# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_inflacion_adj_ipc_undersampled.parquet /home/eanegrin/datasets/

'gsutil' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_inflacion_adj_ipc_undersampled_exp.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
data.shape

(410019, 353)

In [4]:
# corrijo los tipos de estas 2 columnas que se guardaron como string en el parquet

data[['tmobile_app', 'cmobile_app_trx']] = data[['tmobile_app', 'cmobile_app_trx']].astype('float')

In [5]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(391550, 353)

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [8]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [9]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [18]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [19]:
storage_name = "sqlite:///" + db_path + "optimization_experimento_ipc.db"
study_name = "experimento_ipc_v02" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-20 15:27:00,429] Using an existing study with name 'experimento_ipc_v02' instead of creating a new one.


In [20]:
study.optimize(objective, n_trials=50)

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[980]	cv_agg's valid gan_eval: 6.55228e+08 + 5.82237e+06


[I 2024-11-20 15:30:40,575] Trial 1 finished with value: 3276140000.0 and parameters: {'num_leaves': 70, 'learning_rate': 0.019535613904928098, 'min_data_in_leaf': 86, 'feature_fraction': 0.5748671597555829, 'bagging_fraction': 0.6029672848223565}. Best is trial 1 with value: 3276140000.0.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[998]	cv_agg's valid gan_eval: 6.70435e+08 + 7.21188e+06


[I 2024-11-20 15:34:40,426] Trial 2 finished with value: 3352174000.0 and parameters: {'num_leaves': 111, 'learning_rate': 0.04031561089897204, 'min_data_in_leaf': 1572, 'feature_fraction': 0.35570399512329076, 'bagging_fraction': 0.6888488565823928}. Best is trial 2 with value: 3352174000.0.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[999]	cv_agg's valid gan_eval: 6.67911e+08 + 6.58451e+06


...

[I 2024-11-20 18:18:10,329] Trial 48 finished with value: 3348338000.0 and parameters: {'num_leaves': 97, 'learning_rate': 0.047840528901023735, 'min_data_in_leaf': 1027, 'feature_fraction': 0.952148405882902, 'bagging_fraction': 0.676369520663652}. Best is trial 43 with value: 3375106000.0.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.47268e+08 + 5.19534e+06


[I 2024-11-20 18:21:02,509] Trial 49 finished with value: 3236338000.0 and parameters: {'num_leaves': 40, 'learning_rate': 0.015577052751234267, 'min_data_in_leaf': 1651, 'feature_fraction': 0.8202484409111204, 'bagging_fraction': 0.7251158334488714}. Best is trial 43 with value: 3375106000.0.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.67377e+08 + 6.33569e+06


[I 2024-11-20 18:23:50,119] Trial 50 finished with value: 3336886000.0 and parameters: {'num_leaves': 85, 'learning_rate': 0.049891112480013794, 'min_data_in_leaf': 1259, 'feature_fraction': 0.16116938828133726, 'bagging_fraction': 0.845301442121202}. Best is trial 43 with value: 3375106000.0.


Analizamos los resultados as usual

In [21]:
optuna.visualization.plot_optimization_history(study)

In [22]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [23]:
plot_slice(study)

In [24]:
plot_contour(study)

In [25]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [26]:
study.best_trial.params

{'num_leaves': 150,
 'learning_rate': 0.049929102931299305,
 'min_data_in_leaf': 1337,
 'feature_fraction': 0.9913667120903564,
 'bagging_fraction': 0.8195431189437536}

In [27]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

982